### Creates design files for the instrumental learning task
- Stimuli presented
- Reward contingencies
- Cue (SPD/ACC)
- Timing per phase

Symbols are counterbalanced across participants. Trial orders are pseudorandomized: 
- Cues aren't repeated >4 times 
- stimulus sets / symbols aren't repeated >2 times.

In [1]:
import itertools
import numpy as np
import pandas as pd
from pseudorandomizer import Pseudorandomizer

## Set-up phase timing

In [2]:
# timing parameters
def generate_onsets_fixed_duration(phase_settings,
                                   n_trials=100, trial_duration = 9,
                                   n_null_trials=6, null_trial_duration = 6,
                                   null_trial_min_row=8, 
                                   null_trial_max_row=8, 
                                   min_n_rows_separate=8):
    # create onsets/durations dataframe
    trial_nr = np.arange(n_trials, dtype=int)
    
    for i in range(100):
        duration_df = pd.DataFrame(index=trial_nr)

        for phase_nr, phase_dict in enumerate(phase_settings):
            duration_df[phase_dict['phase_name']] = np.random.choice(phase_dict['durations'], 
                                                                     p=phase_dict['probabilities'],
                                                                     replace=True, size=n_trials)

        # make sure first trial, pre-trial iti has at least 1 second
        duration_df.iloc[0,0] = np.max([1, duration_df.iloc[0,0]])
        cumulative_durations = duration_df.apply(sum, axis=1)
        post_iti_durations = trial_duration - cumulative_durations

        # find trials where total trial duration is longer than 9 seconds (trial_duration)
        for idx in np.argwhere(cumulative_durations > trial_duration):
            # subtract the overshoot in trial duration from the phase that takes longest
            longest_phase = duration_df.loc[idx, ['fix_cross_1', 'fix_cross_2', 'fix_cross_3', 'fix_cross_4']].apply(np.argmax, axis=1)
            duration_df.loc[idx, longest_phase] += post_iti_durations[idx].values

        # recalc cumulative duration
        cumulative_duration_nopostiti = duration_df.apply(sum, axis=1)
        duration_df['iti_posttrial'] = trial_duration - cumulative_duration_nopostiti

        if n_null_trials > 0:
            # insert 'null trials'
            good_idx = False
            while not good_idx:
                indx = np.random.choice(np.arange(min_n_rows_separate, 
                                                  duration_df.shape[0]-null_trial_max_row), 
                                        replace=False, size=n_null_trials)
                diffs = np.diff(np.sort(indx))
                if (diffs >= min_n_rows_separate).all():
                    good_idx = True
            duration_df.loc[indx, 'iti_posttrial'] += null_trial_duration
        duration_df['cumulative'] = duration_df.apply(sum, axis=1)
        if np.sum(duration_df<0).sum() > 0 and i == 99:
            print('WARNING! only negative durations found after 100 attempts')
        else:
            break

    # Some reshaping tricks to extract onsets
    onset_df = duration_df.iloc[:,:-1].copy()
    onset_df.index.name = 'trial_nr'
    onset_df = onset_df.reset_index()
    onset_df = onset_df.melt(id_vars=['trial_nr'], value_name='duration', var_name='phase_name')
    
    # make look-up dict
    look_up = {phase_info['phase_name']: phase_nr for phase_nr, phase_info in enumerate(phase_settings)}
    look_up['iti_posttrial'] = len(look_up)
    onset_df['phase_nr'] = onset_df['phase_name'].replace(look_up)
    
    # sort, calculate durations
    onset_df = onset_df.sort_values(by=['trial_nr', 'phase_nr'])
    onset_df['cumulative_duration'] = onset_df['duration'].cumsum()
    onset_df['onset'] = onset_df['cumulative_duration'].shift(1, fill_value=0)
    
    # clean up
    onset_df = onset_df[['trial_nr', 'phase_nr', 'phase_name', 
                         'onset', 'duration', 'cumulative_duration']].reset_index(drop=True)

    return onset_df, duration_df

# onsets, durations = generate_onsets_fixed_duration(phase_settings)

## Counterbalance stimuli across participants

In [3]:
def get_stimuli(subject_id, design, stimulus_sets):
    
    all_stim = counterbalance_stimuli(subject_id, 
                                      all_stimuli=stimulus_sets)
    # flatten
    all_stim = [item for sublist in all_stim for item in sublist]
    
    design['stim_left'] = None
    design['stim_right'] = None
    design['stim_high'] = None
    design['stim_low'] = None

    
    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        design.loc[idx_this_stim, 'stim_high'] = all_stim[i][0]
        design.loc[idx_this_stim, 'stim_low'] = all_stim[i][1]

    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        idx_this_stim_nonrev = (idx_this_stim) & (design.correct_stim_lr == 0)
        idx_this_stim_rev = (idx_this_stim) & (design.correct_stim_lr == 1)

        design.loc[idx_this_stim_nonrev, 'stim_left'] = design.loc[idx_this_stim_nonrev, 'stim_high']
        design.loc[idx_this_stim_nonrev, 'stim_right'] = design.loc[idx_this_stim_nonrev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_left'] = design.loc[idx_this_stim_rev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_right'] = design.loc[idx_this_stim_rev, 'stim_high']
   
    return design

def counterbalance_stimuli(subject_id, all_stimuli=None,
                           reverse_pair=(False, True),
                           shifts_within_blocks=(0,1,2),
                           shifts_across_blocks=(0,1,2),
                           verbose=True):
    """ reverse_pairs = switch first and second half of the sets (useful for different block types) 
    
    """
    import itertools
    from copy import deepcopy
    
    if all_stimuli is None:
        print('WARNING: default stimuli sets used')
        all_stimuli = [[[1, 2], [3, 4], [5, 6]],       # block 1
                       [[7, 8], [9, 10], [11, 12]],    # block 2
                       [[13, 14], [15,16], [17, 18]]]  # block 3

#     options
#     reverse_pair = [False, True]
#     shifts_within_blocks = [0, 1, 2]
#     shifts_across_blocks = [0, 1, 2]

    cb_df = pd.DataFrame(list(itertools.product(shifts_across_blocks,
                                                shifts_within_blocks, 
                                                reverse_pair)),
                         columns=['shifts_across_blocks', 'shifts_within_blocks', 'reverse_pair'])
    n_unique_permutations = cb_df.shape[0]

    pp_zero_based = int(subject_id) - 1
    row_iloc = int(pp_zero_based - np.floor(pp_zero_based / n_unique_permutations) * n_unique_permutations)

    settings = cb_df.iloc[row_iloc]
    stimuli_this_sub = deepcopy(all_stimuli)

    # shift blocks
    for i in range(settings.shifts_across_blocks):
        stimuli_this_sub.insert(len(stimuli_this_sub), stimuli_this_sub.pop(0))

    # loop over stimuli per block
    for block_n in range(len(stimuli_this_sub)):

        # reverse pair order
        for stimulus_set in stimuli_this_sub[block_n]:
            if settings.reverse_pair:
                stimuli_this_sub[block_n] = [x[::-1] for x in stimuli_this_sub[block_n]]  # reverse inner order

        # shift pairs within block
        for i in range(settings.shifts_within_blocks):
            stimuli_this_sub[block_n].insert(len(stimuli_this_sub[block_n]), stimuli_this_sub[block_n].pop(0))

    if verbose:
        print('Stimuli this subject: {}'.format(stimuli_this_sub))
    return stimuli_this_sub

## Combine everything

In [4]:
def generate_block_design(n_trials, ps,
                          stimulus_sets=[0,1,2],
                          include_cue=True,
                          pseudorandomize=True,
                          phase_settings=None, 
                          **kwargs):
    """ Generates design for a single block.
    """
    
    correct_stim_lr = [0, 1]
    if include_cue:
        cues = ['SPD', 'ACC']
    else:
        cues = ['']
    combs = list(itertools.product(stimulus_sets, correct_stim_lr, cues))
    
    # make basic df
    design = pd.DataFrame(combs * int(np.ceil((n_trials/len(combs)))), 
                          columns=['stimulus_set', 'correct_stim_lr', 'cue_txt'])
    
    # randomize / shuffle
    design = design.sample(frac=1).reset_index(drop=True)#.reset_index('trial_ID')
    n_trials_real = design.shape[0]
    design = design.iloc[:n_trials]
    
    if not n_trials == n_trials_real:
        print('WARNING: not totally balanced (%d not a multitude of %d)' %(n_trials, len(combs)))
    
    if pseudorandomize:
        design = Pseudorandomizer(design, max_identical_iters={'stimulus_set': 2,
                                                               'cue_txt': 4}).run()
    
    # Add probabilities (left/right and correct/incorrect - this is redundant, I know)
    design['p_win_left'] = None
    design['p_win_right'] = None
    design['p_win_correct'] = None
    design['p_win_incorrect'] = None
    for stimulus_set, p_win in zip(stimulus_sets, ps):
        p_win_high, p_win_low = p_win[0], p_win[1]
        design.loc[(design.stimulus_set==stimulus_set) & (design.correct_stim_lr == 0), 'p_win_left'] = p_win_high
        design.loc[(design.stimulus_set==stimulus_set) & (design.correct_stim_lr == 1), 'p_win_right'] = p_win_high
        design.loc[(design.stimulus_set==stimulus_set) & (design.correct_stim_lr == 1), 'p_win_left'] = p_win_low
        design.loc[(design.stimulus_set==stimulus_set) & (design.correct_stim_lr == 0), 'p_win_right'] = p_win_low
        design.loc[(design.stimulus_set==stimulus_set), 'p_win_correct'] = p_win_high
        design.loc[(design.stimulus_set==stimulus_set), 'p_win_incorrect'] = p_win_low
    
    design['trial_nr'] = np.arange(design.shape[0], dtype='int')
    
    if phase_settings is not None:
#         onsets_long, _ = generate_onsets_fixed_duration(phase_settings=phase_settings, n_trials=n_trials)
#         onsets_wide = pd.pivot_table(onsets_long, values='onset', index='trial_nr', columns='phase_name').reset_index()
        onsets, durations = generate_onsets_fixed_duration(phase_settings=phase_settings, n_trials=n_trials, **kwargs)
        durations['trial_nr'] = np.arange(durations.shape[0], dtype='int')
        durations = durations.rename(columns={'cumulative': 'total_duration'})
        design = pd.merge(design, durations, on='trial_nr')
        
        # make sure to generate the onsets long file
        acc_trials = design.loc[design['cue_txt'] == 'ACC', 'trial_nr']
        spd_trials = design.loc[design['cue_txt'] == 'SPD', 'trial_nr']
        
        onsets.loc[(np.in1d(onsets['trial_nr'], acc_trials) & (onsets.phase_name == 'cue')), 'phase_name'] = 'cue_acc'
        onsets.loc[(np.in1d(onsets['trial_nr'], spd_trials) & (onsets.phase_name == 'cue')), 'phase_name'] = 'cue_spd'
        
        # clean up
        design = design[['trial_nr', 'stimulus_set', 'correct_stim_lr', 'cue_txt', 'p_win_left', 'p_win_right', 'p_win_correct', 'p_win_incorrect',
                         'fix_cross_1', 'cue', 'fix_cross_2', 'stimulus', 'fix_cross_3', 
                         'highlight', 'fix_cross_4', 'feedback', 'iti_posttrial', 'total_duration']]
        return design, onsets
    
    return design

# design, onsets = generate_block_design(n_trials=96, 
#                                        stimulus_sets=[0, 1, 2], 
#                                        ps=[[0.8, 0.2], [0.7, 0.3], [0.6, 0.4]],
#                                        phase_settings=phase_settings)

## Session 1: Agathodaimon alphabet
Just enter normal characters, they're presented in an Agathodaimon font

In [8]:
all_stimulus_sets = [[['C', 'c'],     #Cc
                      ['D', 'g'],     #Dg
                      ['f', 'x']],    #fx
                     [['j', 'N'],     #jN
                      ['Q', 'J'],     #QJ
                      ['m', 'M']],    #mM
                     [['R', 'S'],     #RS
                      ['s', 'T'],     #sT
                      ['t', 'U']]]    #tU


n_runs = 3
subject_id = 1

## TIMING FOR BEHAVIORAL PILOT: No fix cross 2, 3, 4
pseudoexponential_ps = np.array([8, 4, 2, 1])
pseudoexponential_ps = pseudoexponential_ps/pseudoexponential_ps.sum()
phase_settings = [{'phase_name': 'fix_cross_1',
                   'durations': [0.5],
                   'probabilities': [1]},
                  {'phase_name': 'cue',
                   'durations': [0.75],
                   'probabilities': [1]},
                  {'phase_name': 'fix_cross_2',
                   'durations': [0],
                   'probabilities': [1]},
                  {'phase_name': 'stimulus',
                   'durations': [1.5],
                   'probabilities': [1]},
                  {'phase_name': 'fix_cross_3',
                   'durations': [0],
                   'probabilities': [1]},
                  {'phase_name': 'highlight',
                   'durations': [0.5],
                   'probabilities': [1]},
                  {'phase_name': 'fix_cross_4',
                   'durations': [0],
                   'probabilities': [1]},
                  {'phase_name': 'feedback',
                   'durations': [.75],
                   'probabilities': [1]}]

In [14]:
# WARNING: SETTINGS FOR BEHAVIORAL PILOT
n_null_trials = 0
n_trials = 12*9  # must be a product of 12 to ensure counterbalanced stimuli
n_subs = 101

for subject_id in np.arange(1, n_subs+1, dtype=int):
    designs_this_session = []
    for run in range(1,n_runs+1):
        s0_this_block = (run-1)*3
        stimulus_sets = [s0_this_block, s0_this_block+1, s0_this_block+2]
        design, onsets = generate_block_design(n_trials = n_trials, 
                                               ps=[[0.8, 0.2], [0.7, 0.3], [0.6, 0.4]],
                                               stimulus_sets=stimulus_sets,  # dont touch this
                                               phase_settings=phase_settings,
                                               n_null_trials=n_null_trials)
        design['block'] = run
        design['iti_posttrial'] = 0  # sets all post-trial ITIs to 0
        designs_this_session.append(design)

    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design, stimulus_sets=all_stimulus_sets)

    ###
    subject_id_str = str(subject_id).zfill(3)
    design.to_csv('../designs_SAT-learning/sub-{}_ses-1_task-SAT-learning_design.csv'.format(subject_id_str), sep='\t')

/Users/steven/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


Stimuli this subject: [[['a', 'b'], ['C', 'D'], ['E', 'F']], [['g', 'i'], ['J', 'n'], ['O', 'p']], [['3', 'Ç'], ['r', 's'], ['u', 'w']]]
Stimuli this subject: [[['b', 'a'], ['D', 'C'], ['F', 'E']], [['i', 'g'], ['n', 'J'], ['p', 'O']], [['Ç', '3'], ['s', 'r'], ['w', 'u']]]
Stimuli this subject: [[['C', 'D'], ['E', 'F'], ['a', 'b']], [['J', 'n'], ['O', 'p'], ['g', 'i']], [['r', 's'], ['u', 'w'], ['3', 'Ç']]]
Stimuli this subject: [[['D', 'C'], ['F', 'E'], ['b', 'a']], [['n', 'J'], ['p', 'O'], ['i', 'g']], [['s', 'r'], ['w', 'u'], ['Ç', '3']]]
Stimuli this subject: [[['E', 'F'], ['a', 'b'], ['C', 'D']], [['O', 'p'], ['g', 'i'], ['J', 'n']], [['u', 'w'], ['3', 'Ç'], ['r', 's']]]
Stimuli this subject: [[['F', 'E'], ['b', 'a'], ['D', 'C']], [['p', 'O'], ['i', 'g'], ['n', 'J']], [['w', 'u'], ['Ç', '3'], ['s', 'r']]]
Stimuli this subject: [[['g', 'i'], ['J', 'n'], ['O', 'p']], [['3', 'Ç'], ['r', 's'], ['u', 'w']], [['a', 'b'], ['C', 'D'], ['E', 'F']]]
Stimuli this subject: [[['i', 'g'], ['n',

Stimuli this subject: [[['g', 'i'], ['J', 'n'], ['O', 'p']], [['3', 'Ç'], ['r', 's'], ['u', 'w']], [['a', 'b'], ['C', 'D'], ['E', 'F']]]
Stimuli this subject: [[['i', 'g'], ['n', 'J'], ['p', 'O']], [['Ç', '3'], ['s', 'r'], ['w', 'u']], [['b', 'a'], ['D', 'C'], ['F', 'E']]]
Stimuli this subject: [[['J', 'n'], ['O', 'p'], ['g', 'i']], [['r', 's'], ['u', 'w'], ['3', 'Ç']], [['C', 'D'], ['E', 'F'], ['a', 'b']]]
Stimuli this subject: [[['n', 'J'], ['p', 'O'], ['i', 'g']], [['s', 'r'], ['w', 'u'], ['Ç', '3']], [['D', 'C'], ['F', 'E'], ['b', 'a']]]
Stimuli this subject: [[['O', 'p'], ['g', 'i'], ['J', 'n']], [['u', 'w'], ['3', 'Ç'], ['r', 's']], [['E', 'F'], ['a', 'b'], ['C', 'D']]]
Stimuli this subject: [[['p', 'O'], ['i', 'g'], ['n', 'J']], [['w', 'u'], ['Ç', '3'], ['s', 'r']], [['F', 'E'], ['b', 'a'], ['D', 'C']]]
Stimuli this subject: [[['3', 'Ç'], ['r', 's'], ['u', 'w']], [['a', 'b'], ['C', 'D'], ['E', 'F']], [['g', 'i'], ['J', 'n'], ['O', 'p']]]
Stimuli this subject: [[['Ç', '3'], ['s',

## Session 2: Galgolitic
Unfortunately unicode characters 0C2xx aren't displayed well in Psychopy3, so we use a specific font here that maps Latin characters to Galgolitic characters

In [11]:
all_stimulus_sets = [[['a', 'b'],     # Ⰰ Ⱀ
                      ['C', 'D'],     # Ⱐ Ⰱ
                      ['E', 'F']],    # fx
                     [['g', 'i'],     #jN
                      ['J', 'n'],     #QJ
                      ['O', 'p']],    #mM
                     [['3', 'Ç'],     #RS
                      ['r', 's'],     #sT
                      ['u', 'w']]]    #tU
                     

In [15]:
# WARNING: SETTINGS FOR BEHAVIORAL PILOT
n_null_trials = 0
n_trials = 12*9  # must be a product of 12 to ensure counterbalanced stimuli
n_subs = 101

for subject_id in np.arange(1, n_subs+1, dtype=int):
    designs_this_session = []
    for run in range(1,n_runs+1):
        s0_this_block = (run-1)*3
        stimulus_sets = [s0_this_block, s0_this_block+1, s0_this_block+2]
        design, onsets = generate_block_design(n_trials = n_trials, 
                                               ps=[[0.8, 0.2], [0.7, 0.3], [0.6, 0.4]],
                                               stimulus_sets=stimulus_sets,  # dont touch this
                                               phase_settings=phase_settings,
                                               n_null_trials=n_null_trials)
        design['block'] = run
        design['iti_posttrial'] = 0  # sets all post-trial ITIs to 0
        designs_this_session.append(design)

    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design, stimulus_sets=all_stimulus_sets)

    ###
    subject_id_str = str(subject_id).zfill(3)
    design.to_csv('../designs_SAT-learning/sub-{}_ses-2_task-SAT-learning_design.csv'.format(subject_id_str), sep='\t')

Stimuli this subject: [[['a', 'b'], ['C', 'D'], ['E', 'F']], [['g', 'i'], ['J', 'n'], ['O', 'p']], [['3', 'Ç'], ['r', 's'], ['u', 'w']]]
Stimuli this subject: [[['b', 'a'], ['D', 'C'], ['F', 'E']], [['i', 'g'], ['n', 'J'], ['p', 'O']], [['Ç', '3'], ['s', 'r'], ['w', 'u']]]
Stimuli this subject: [[['C', 'D'], ['E', 'F'], ['a', 'b']], [['J', 'n'], ['O', 'p'], ['g', 'i']], [['r', 's'], ['u', 'w'], ['3', 'Ç']]]
Stimuli this subject: [[['D', 'C'], ['F', 'E'], ['b', 'a']], [['n', 'J'], ['p', 'O'], ['i', 'g']], [['s', 'r'], ['w', 'u'], ['Ç', '3']]]
Stimuli this subject: [[['E', 'F'], ['a', 'b'], ['C', 'D']], [['O', 'p'], ['g', 'i'], ['J', 'n']], [['u', 'w'], ['3', 'Ç'], ['r', 's']]]
Stimuli this subject: [[['F', 'E'], ['b', 'a'], ['D', 'C']], [['p', 'O'], ['i', 'g'], ['n', 'J']], [['w', 'u'], ['Ç', '3'], ['s', 'r']]]
Stimuli this subject: [[['g', 'i'], ['J', 'n'], ['O', 'p']], [['3', 'Ç'], ['r', 's'], ['u', 'w']], [['a', 'b'], ['C', 'D'], ['E', 'F']]]
Stimuli this subject: [[['i', 'g'], ['n',

Stimuli this subject: [[['g', 'i'], ['J', 'n'], ['O', 'p']], [['3', 'Ç'], ['r', 's'], ['u', 'w']], [['a', 'b'], ['C', 'D'], ['E', 'F']]]
Stimuli this subject: [[['i', 'g'], ['n', 'J'], ['p', 'O']], [['Ç', '3'], ['s', 'r'], ['w', 'u']], [['b', 'a'], ['D', 'C'], ['F', 'E']]]
Stimuli this subject: [[['J', 'n'], ['O', 'p'], ['g', 'i']], [['r', 's'], ['u', 'w'], ['3', 'Ç']], [['C', 'D'], ['E', 'F'], ['a', 'b']]]
Stimuli this subject: [[['n', 'J'], ['p', 'O'], ['i', 'g']], [['s', 'r'], ['w', 'u'], ['Ç', '3']], [['D', 'C'], ['F', 'E'], ['b', 'a']]]
Stimuli this subject: [[['O', 'p'], ['g', 'i'], ['J', 'n']], [['u', 'w'], ['3', 'Ç'], ['r', 's']], [['E', 'F'], ['a', 'b'], ['C', 'D']]]
Stimuli this subject: [[['p', 'O'], ['i', 'g'], ['n', 'J']], [['w', 'u'], ['Ç', '3'], ['s', 'r']], [['F', 'E'], ['b', 'a'], ['D', 'C']]]
Stimuli this subject: [[['3', 'Ç'], ['r', 's'], ['u', 'w']], [['a', 'b'], ['C', 'D'], ['E', 'F']], [['g', 'i'], ['J', 'n'], ['O', 'p']]]
Stimuli this subject: [[['Ç', '3'], ['s',

In [ ]:
# print(u"PsychoPy \u00A9Jon Peirce")

In [ ]:
tmp = design[['fix_cross_1', 'cue', 'fix_cross_2', 'stimulus', 'fix_cross_3', 'highlight', 'fix_cross_4', 'feedback', 'iti_posttrial']]
tmp.head()

In [ ]:
def design_to_onsets(design, phase_settings):
#     design.index.name = 'trial_nr'
#     design = design.reset_index()

#     phase_names = [x['phase_name'] for x in phase_settings]
#     phase_names.append('iti_posttrial')

    onset_df = design.copy()
    onset_df = onset_df.melt(id_vars=['trial_nr'], 
                             value_name='duration', 
                             var_name='phase_name')
    
    look_up = {phase_info['phase_name']: phase_nr for phase_nr, phase_info in enumerate(phase_settings)}
    look_up['iti_posttrial'] = len(look_up)
    onset_df['phase_nr'] = onset_df['phase_name'].replace(look_up)
    
    # sort, calculate durations
    onset_df = onset_df.sort_values(by=['trial_nr', 'phase_nr'])
    onset_df['cumulative_duration'] = onset_df['duration'].cumsum()
    onset_df['onset'] = onset_df['cumulative_duration'].shift(1, fill_value=0)
    
    # clean up
    onset_df = onset_df[['trial_nr', 'phase_nr', 'phase_name', 
                         'onset', 'duration', 'cumulative_duration']].reset_index(drop=True)
    
    return onset_df

block_idx = design.block == 1
cols = ['trial_nr', 'fix_cross_1', 'cue', 'fix_cross_2', 'stimulus', 'fix_cross_3', 'highlight', 'fix_cross_4', 'feedback', 'iti_posttrial']

preset_onsets = design_to_onsets(design.loc[block_idx, cols], 
                                 phase_settings=phase_settings)
preset_onsets = preset_onsets.rename(columns = {'phase_name': 'event_type'})
preset_onsets['event_type'] = preset_onsets['event_type'].replace({'iti_posttrial': 'fix_cross_5'})

In [ ]:
import os
import pandas as pd

In [ ]:
path = '/Users/steven/Sync/PhDprojects/IMCN-learningtask/IMCN-learningtask/data'
fn = 'sub-1_task-learning_datetime-20191008-111056_events.tsv'
measured = pd.read_csv(os.path.join(path, fn), sep='\t')
measured_onsets = measured.loc[pd.notnull(measured['duration']) & (measured['trial_nr'] >= 0.), ['trial_nr', 'onset', 'event_type', 'duration']]

In [ ]:
merged = pd.merge(preset_onsets, measured_onsets, on=['trial_nr', 'event_type'], suffixes=('_preset', '_measured'))
merged['onset_measured_reset'] = merged['onset_measured'] - merged.iloc[0]['onset_measured']
merged['difference'] = merged['onset_measured_reset'] - merged['onset_preset']

In [ ]:
merged

In [ ]:
tmp = np.array([1, 2, 3, 4])

np.maximum(tmp, 2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dat = np.loadtxt('/Volumes/SteefSSD/IMCN-learningtask/IMCN-learningtask/data/sub-1_task-learning_datetime-20191016-131043_frameintervals.log', delimiter=',')

In [ ]:
f, ax = plt.subplots()
ax.plot(dat)

ax.set_ylim(0, 0.1)
plt.gcf().set_size_inches(10, 5)

In [ ]:
data = pd.read_csv('/Volumes/SteefSSD/IMCN-learningtask/IMCN-learningtask/data/sub-1_task-learning_datetime-20191016-131043_events.tsv', sep='\t')

In [ ]:
data.shape

In [ ]:
data

In [ ]:
data.loc[data.event_type == 'stimulus', 'nr_frames']

In [ ]:
plt.hist(dat.round(5)[2:], bins=20)

In [ ]:
1/120

In [ ]:
dat2 = dat[2:]
np.mean(dat2>0.02)

In [ ]:
dat[2:].min()

In [ ]:
# def counterbalance_stimuli(subject_id, all_sets):
#     """ switch sets = switch first and second half of the sets (useful for different block types) """
#     import itertools
#     from copy import deepcopy
    
#     n_sets = len(all_sets)
#     n_shifts = [0, 3, 6]  #np.arange(n_sets, dtype=int) #[0, 1, 2, 3, 4, 5, 6, 7, 8]
#     rev_inner = [False, True]
#     n_shifts_inner = [0, 1, 2]

#     cb_df = pd.DataFrame(list(itertools.product(n_shifts, rev_inner)),
#                          columns=['n_shifts', 'rev_inner'])
#     n_unique_permutations = cb_df.shape[0]
    
#     cb_df['pp'] = np.arange(1, cb_df.shape[0] + 1)
#     for stim_n in range(1, n_sets*2+1):  # loop over *stimuli*
#         cb_df['stim_%d' % stim_n] = None

#     for pp in cb_df['pp']:
#         idx = cb_df.pp == pp
#         reverse_inner = cb_df.loc[idx, 'rev_inner'].iloc[0]
#         n_shifts = cb_df.loc[idx, 'n_shifts'].iloc[0]

#         sets = deepcopy([all_sets])
    
#         sets_allocated = 0
#         for set_n, set_ in enumerate(sets):
#             for i in range(n_shifts):
#                 set_.insert(len(set_), set_.pop(0))  # move first item to last place
            
#             if reverse_inner:
#                 set_ = [x[::-1] for x in set_]  # reverse inner order

# #             print('pp %d, %d, %s' % (pp, set_n, set_))
#             #### NB: you could just use set_ as a final result; the placing in the dataframe and then reverting
#             # back to a nested list is definitely not necessary but may help clarify what's going on here...
#             for to_allocate in range(n_sets):
#                 for to_allocate_i in [0, 1]:
#                     cb_df.loc[idx, 'stim_%d' % (sets_allocated + 1)] = set_[to_allocate][to_allocate_i]
#                     sets_allocated += 1

#     pp_zero_based = int(subject_id) - 1
#     row_iloc = int(pp_zero_based - np.floor(pp_zero_based / n_unique_permutations) * n_unique_permutations)
#     colnames = cb_df.columns
#     stim_list = cb_df.iloc[row_iloc][[x for x in colnames if 'stim' in x]].values.tolist()
#     stim_nested_list = [[stim_list[0 + y * 2], stim_list[1 + y * 2]] for y in range(int(n_sets))]
#     print('Stimuli/set order for this pp: %s' % stim_nested_list)
#     return stim_nested_list



In [ ]:
import unicodedata

In [ ]:
repr('Ⱝ')

In [ ]:
'Андрей'

In [ ]:
repr('Ⱝ')

In [ ]:
print((u'2C00'))